![](banner.png)
# Objetivos
+ Cargar los datos de una base de datos oltp y hacer las transformaciones necesarias para convertir esa información al modelo dimensional

# Carga de librerias

In [1]:
#Librerias para trabajar los datos
import pandas as pd
import numpy as np

In [2]:
#Carga de los datos principales
fact_table = pd.read_csv('generated/data_deptos.csv',sep=',',index_col=False,low_memory=False)

# Creación de dimensiones y tabla de hechos
Es necesario que guardemos la fact table con ; para que power bi lea, so cambiaremos los ".", por ","

In [3]:
fact_table.head()

,sexo,anio,edad,codigo_depto,nombre_depto,codigo_evento,evento,c/1000,c/10000,c/100000
0,Hombres,2010,0-4 años,5,Antioquia,101,ENFERMEDADES INFECCIOSAS INTESTINALES,0.00386,0.03859,0.38594
1,Hombres,2010,0-4 años,5,Antioquia,103,CIERTAS ENF. TRANSMITIDAS POR VECTORES Y RABIA,0.00386,0.03859,0.38594
2,Hombres,2010,0-4 años,5,Antioquia,105,MENINGITIS,0.00772,0.07719,0.77187
3,Hombres,2010,0-4 años,5,Antioquia,106,SEPTICEMIA,0.00386,0.03859,0.38594
4,Hombres,2010,0-4 años,5,Antioquia,109,INFECCIONES RESPIRATORIAS AGUDAS,0.03473,0.34734,3.47343


In [4]:
#primero lo "desarreglamos"
fact_table["c/1000"] = fact_table["c/1000"].apply(lambda x : str(x))
fact_table["c/10000"] = fact_table["c/10000"].apply(lambda x : str(x))
fact_table["c/100000"] = fact_table["c/100000"].apply(lambda x : str(x))

#Arreglamos pa que todo quede con comas
fact_table["c/1000"] = fact_table["c/1000"].apply(lambda x : x.replace(".",","))
fact_table["c/10000"] = fact_table["c/10000"].apply(lambda x : x.replace(".",","))
fact_table["c/100000"] = fact_table["c/100000"].apply(lambda x : x.replace(".",","))

## Creación dimensión Fecha

In [5]:
#Creamos el dataframe
df_fecha = pd.DataFrame(fact_table["anio"].unique())
df_fecha = df_fecha.reset_index()
df_fecha.columns = ["fecha_key", "anio"]
df_fecha["fecha_key"] = df_fecha["fecha_key"]+1

#Actualizamos la tabla principal
fact_table = pd.merge(fact_table, df_fecha, on="anio")
fact_table = fact_table.drop(columns=['anio'])

In [6]:
fact_table.head(5)

,sexo,edad,codigo_depto,nombre_depto,codigo_evento,evento,c/1000,c/10000,c/100000,fecha_key
0,Hombres,0-4 años,5,Antioquia,101,ENFERMEDADES INFECCIOSAS INTESTINALES,"0,00386","0,03859","0,38594",1
1,Hombres,0-4 años,5,Antioquia,103,CIERTAS ENF. TRANSMITIDAS POR VECTORES Y RABIA,"0,00386","0,03859","0,38594",1
2,Hombres,0-4 años,5,Antioquia,105,MENINGITIS,"0,00772","0,07719","0,77187",1
3,Hombres,0-4 años,5,Antioquia,106,SEPTICEMIA,"0,00386","0,03859","0,38594",1
4,Hombres,0-4 años,5,Antioquia,109,INFECCIONES RESPIRATORIAS AGUDAS,"0,03473","0,34734","3,47343",1


In [7]:
#Guardamos la dimensión fecha
df_fecha.to_csv('generated/dim_fecha.csv',index = False)

## Creación dimensión Perfil

In [8]:
#obtengo edades y sexo unicas, luego creo a para hacer un join sin parametro
list_edad = pd.DataFrame(fact_table["edad"].unique())
list_edad["a"] = "a"
list_sexo = pd.DataFrame(fact_table["sexo"].unique())
list_sexo["a"] = "a"

In [9]:
r1 = pd.merge(list_edad, list_sexo, on="a")
r1.head(5)

,0_x,a,0_y
0,0-4 años,a,Hombres
1,0-4 años,a,Mujeres
2,15-44 años,a,Hombres
3,15-44 años,a,Mujeres
4,45-64 años,a,Hombres


In [10]:
#Creamos el dataframe
df_perfil = r1.copy()
df_perfil = df_perfil.drop(columns=['a'])
df_perfil = df_perfil.reset_index()
df_perfil.columns = ["perfil_key","edad", "sexo"]
df_perfil["perfil_key"] = df_perfil["perfil_key"]+1

In [11]:
df_perfil.head(5)

,perfil_key,edad,sexo
0,1,0-4 años,Hombres
1,2,0-4 años,Mujeres
2,3,15-44 años,Hombres
3,4,15-44 años,Mujeres
4,5,45-64 años,Hombres


In [12]:
#Actualizamos la tabla principal
fact_table = pd.merge(fact_table, df_perfil,  on=['edad','sexo'])
fact_table = fact_table.drop(columns=['edad','sexo'])
fact_table

,codigo_depto,nombre_depto,codigo_evento,evento,c/1000,c/10000,c/100000,fecha_key,perfil_key
0,5,Antioquia,101,ENFERMEDADES INFECCIOSAS INTESTINALES,"0,00386","0,03859","0,38594",1,1
1,5,Antioquia,103,CIERTAS ENF. TRANSMITIDAS POR VECTORES Y RABIA,"0,00386","0,03859","0,38594",1,1
2,5,Antioquia,105,MENINGITIS,"0,00772","0,07719","0,77187",1,1
3,5,Antioquia,106,SEPTICEMIA,"0,00386","0,03859","0,38594",1,1
4,5,Antioquia,109,INFECCIONES RESPIRATORIAS AGUDAS,"0,03473","0,34734","3,47343",1,1
...,...,...,...,...,...,...,...,...,...
74218,99,Vichada,302,ENFERMEDADES HIPERTENSIVAS,"0,2004","2,00401","20,04008",8,10
74219,99,Vichada,306,INSUFICIENCIA CARDIACA,"0,2004","2,00401","20,04008",8,10
74220,99,Vichada,307,ENFERMEDADES CEREBROVASCULARES,"0,2004","2,00401","20,04008",8,10
74221,99,Vichada,605,ENF. CRONICAS VIAS REPIRATORIAS INFERIORES,"0,4008","4,00802","40,08016",8,10


In [13]:
#Guardamos la dimensión perfil
df_perfil.to_csv('generated/dim_perfil.csv',index = False)

## Creación dimensión Lugar

In [14]:
df_temp = fact_table.copy()

In [15]:
df_lugar = df_temp[["codigo_depto","nombre_depto"]]

In [16]:
df_lugar = df_lugar.drop_duplicates()

In [17]:
#Creamos el dataframe
df_lugar = df_lugar.reset_index()
df_lugar.columns = ["lugar_key","codigo_depto","nombre_depto"]
df_lugar["lugar_key"] = df_lugar["lugar_key"]+1

In [18]:
#Para departamentos a añadir las latitudes y las longitudes
depto = pd.read_csv('sources/colombia_depto.csv', header=None, encoding='utf8') 
depto.columns = ["codigo_depto","nombre_depto","lati_depto", "long_depto"]
depto = depto.drop(columns=["nombre_depto"])

In [19]:
df_lugar = pd.merge(df_lugar, depto, on='codigo_depto')

In [20]:
#luego lo "desarreglamos"
df_lugar["lati_depto"] = df_lugar["lati_depto"].apply(lambda x : str(x))
df_lugar["long_depto"] = df_lugar["long_depto"].apply(lambda x : str(x))

#Arreglamos pa que todo quede con comas
df_lugar["lati_depto"] = df_lugar["lati_depto"].apply(lambda x : x.replace(".",","))
df_lugar["long_depto"] = df_lugar["long_depto"].apply(lambda x : x.replace(".",","))

In [21]:
df_lugar.head()

,lugar_key,codigo_depto,nombre_depto,lati_depto,long_depto
0,1,5,Antioquia,"7,0","-75,5"
1,61,8,Atlántico,"10,75","-75,0"
2,107,11,Bogotá,"4,6","-74,0833333"
3,166,13,Bolívar,"9,0","-74,3333333"
4,210,15,Boyacá,"5,5","-72,5"


In [22]:
#Actualizamos la tabla principal
fact_table = pd.merge(fact_table, df_lugar,  on=['nombre_depto'], suffixes = ("_l","_r"))
fact_table = fact_table.drop(columns=['codigo_depto_l', 'codigo_depto_r', 'nombre_depto','lati_depto','long_depto'])
fact_table

,codigo_evento,evento,c/1000,c/10000,c/100000,fecha_key,perfil_key,lugar_key
0,101,ENFERMEDADES INFECCIOSAS INTESTINALES,"0,00386","0,03859","0,38594",1,1,1
1,103,CIERTAS ENF. TRANSMITIDAS POR VECTORES Y RABIA,"0,00386","0,03859","0,38594",1,1,1
2,105,MENINGITIS,"0,00772","0,07719","0,77187",1,1,1
3,106,SEPTICEMIA,"0,00386","0,03859","0,38594",1,1,1
4,109,INFECCIONES RESPIRATORIAS AGUDAS,"0,03473","0,34734","3,47343",1,1,1
...,...,...,...,...,...,...,...,...
74218,302,ENFERMEDADES HIPERTENSIVAS,"0,2004","2,00401","20,04008",8,10,1079
74219,306,INSUFICIENCIA CARDIACA,"0,2004","2,00401","20,04008",8,10,1079
74220,307,ENFERMEDADES CEREBROVASCULARES,"0,2004","2,00401","20,04008",8,10,1079
74221,605,ENF. CRONICAS VIAS REPIRATORIAS INFERIORES,"0,4008","4,00802","40,08016",8,10,1079


In [23]:
#Guardamos la dimensión lugar
df_lugar.to_csv('generated/dim_lugar.csv',index = False, sep = ";")

## Creación dimensión Evento

In [24]:
df_temp = fact_table.copy()

In [25]:
df_evento = df_temp[["codigo_evento","evento"]]

In [26]:
df_evento = df_evento.drop_duplicates()

In [27]:
#Creamos el dataframe
df_evento = df_evento.reset_index()
df_evento.columns = ["evento_key","codigo_evento","evento"]
df_evento["evento_key"] = df_evento["evento_key"]+1

In [28]:
#Actualizamos la tabla principal
fact_table = pd.merge(fact_table, df_evento, on=["evento","codigo_evento"])
fact_table = fact_table.drop(columns=['evento', "codigo_evento"])

In [29]:
fact_table

,c/1000,c/10000,c/100000,fecha_key,perfil_key,lugar_key,evento_key
0,"0,00386","0,03859","0,38594",1,1,1,1
1,"0,02316","0,23156","2,31562",1,1,1,1
2,"0,01581","0,1581","1,581",2,1,1,1
3,"0,02767","0,27667","2,76674",2,1,1,1
4,"0,00401","0,04008","0,40084",3,1,1,1
...,...,...,...,...,...,...,...
74218,"0,15931","1,59305","15,93054",4,4,1079,2070
74219,"0,15818","1,58184","15,8184",5,4,1079,2070
74220,"0,35284","3,52844","35,28443",6,4,1079,2070
74221,"0,07739","0,77393","7,73934",7,4,1079,2070


In [30]:
#Guardamos la dimensión fecha
df_evento.to_csv('generated/dim_evento.csv',index = False)

In [31]:
#Antes de guardar la tabla de hechos
fact_table=fact_table.reset_index()
fact_table.columns = ["fact_key", "c_over_1000", "c_over_10000", "c_over_100000","fecha_key","perfil_key","lugar_key","evento_key"]

In [32]:
#Guardamos la tabla de hechos
fact_table.to_csv('generated/fact_table.csv',index = False, sep = ";")